In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import requests
import os
import time
# import re # Descomente se precisar de regex mais complexas no futuro

# --- Configurações do Usuário ---

# Opção 1: Definir a API Key diretamente (recomendado se não estiver usando Colab/Drive)
API_KEY = "XfCz461qd3BMH8NDt0GRWLjgUycbOs5k" # SUBSTITUA PELA SUA CHAVE DE API REAL

# Opção 2: Ler a API Key do Google Drive (descomente as linhas abaixo se estiver usando Colab)
# from google.colab import drive
# drive.mount('/content/drive')
# API_KEY_FILE_PATH = 'drive/MyDrive/apikey.txt' # Ajuste o caminho conforme necessário
# try:
#     with open(API_KEY_FILE_PATH) as keyfile:
#         API_KEY = keyfile.read().strip()
#     if not API_KEY:
#         print(f"ERRO: Arquivo de API Key '{API_KEY_FILE_PATH}' está vazio.")
#         exit()
#     print("API Key carregada do Google Drive.")
# except FileNotFoundError:
#     print(f"ERRO: Arquivo de API Key '{API_KEY_FILE_PATH}' não encontrado. Defina a API_KEY diretamente no script.")
#     exit()

if API_KEY == "SUA_API_KEY_AQUI" or not API_KEY:
    print("ERRO CRÍTICO: A API_KEY não foi definida. "
          "Por favor, edite o script para adicionar sua chave ou configure o caminho para o arquivo no Google Drive.")
    exit()

# Caminho para sua planilha CSV de entrada
PLANILHA_ENTRADA = "/work/CenaNoink/planilha30_31_32(Sheet1).csv" # AJUSTE SE NECESSÁRIO

# Nomes das colunas na planilha de entrada
COLUNA_DOI = "DOI"
COLUNA_TERMO_BUSCA_FULLTEXT = "TERMO_BUSCA_FULLTEXT" # IMPORTANTE: Adicione esta coluna à sua planilha

# Arquivo CSV de saída para os resultados
ARQUIVO_SAIDA_RESULTADOS = "resultados_busca_fulltext_core_v2_ref.csv"

# Configurações para o retry em caso de rate limit (erro 429)
MAX_RETRIES_API = 3
INITIAL_BACKOFF_SECONDS_API = 65 # Levemente aumentado
MAX_BACKOFF_SECONDS_API = 360 # Levemente aumentado

# Delay base entre o processamento de cada linha do CSV (em segundos)
DELAY_ENTRE_LINHAS_CSV = 8

def executar_query_api_core(base_url, query_params, headers, max_retries, initial_backoff, max_backoff):
    """
    Executa uma chamada à API CORE com tratamento de erro, incluindo rate limits (429) e retries.
    Retorna o objeto de resposta da API ou None em caso de falha persistente.
    """
    current_retry = 0
    backoff_time = initial_backoff

    while current_retry <= max_retries:
        try:
            if current_retry > 0:
                print(f"Nova tentativa ({current_retry}/{max_retries}) para URL: {base_url} com params: {query_params.get('q')}")
            
            response = requests.get(base_url, headers=headers, params=query_params, timeout=45)
            
            if response.status_code == 429: # Too Many Requests
                print(f"Erro 429 (Too Many Requests) para URL: {response.url}.")
                retry_after_header = response.headers.get("Retry-After")
                wait_time = backoff_time
                if retry_after_header:
                    try:
                        wait_time = int(retry_after_header)
                        print(f"Cabeçalho Retry-After: {wait_time}s.")
                    except ValueError:
                        print(f"Retry-After ('{retry_after_header}') inválido. Usando backoff: {wait_time}s.")
                else:
                    print(f"Sem Retry-After. Usando backoff: {wait_time}s.")
                
                wait_time = min(max(wait_time, 1), max_backoff) # Garante espera mínima e não excede máxima
                
                if current_retry < max_retries:
                    print(f"Aguardando {wait_time} segundos antes da próxima tentativa...")
                    time.sleep(wait_time)
                    current_retry += 1
                    backoff_time = min(backoff_time * 1.5, max_backoff) # Aumento para próxima tentativa
                    continue # Tenta novamente
                else:
                    print(f"Máximo de retries ({max_retries}) atingido após erro 429 para {response.url}.")
                    return response # Retorna a última resposta de erro 429

            response.raise_for_status() # Levanta HTTPError para outros códigos 4xx/5xx
            return response # Retorna a resposta bem-sucedida

        except requests.exceptions.HTTPError as e:
            print(f"Erro HTTP {e.response.status_code} para URL {e.response.url}: {e.response.text[:300]}")
            # Para erros como 400, 401, 403, 404 (que não sejam 429), geralmente não adianta tentar novamente
            # A menos que seja um erro de servidor (5xx) que pode ser temporário
            if e.response.status_code >= 500 and current_retry < max_retries :
                 print(f"Erro de servidor detectado. Aguardando {backoff_time}s antes de tentar novamente...")
                 time.sleep(backoff_time)
                 current_retry +=1
                 backoff_time = min(backoff_time * 1.5, max_backoff)
                 continue
            return e.response # Retorna a resposta de erro
        
        except requests.exceptions.RequestException as e: # Erros de conexão, timeout, etc.
            print(f"Erro de requisição para URL {base_url} (Tentativa {current_retry + 1}): {e}")
            if current_retry < max_retries:
                print(f"Aguardando {backoff_time} segundos antes de tentar novamente...")
                time.sleep(backoff_time)
                current_retry += 1
                backoff_time = min(backoff_time * 1.5, max_backoff)
            else:
                print(f"Máximo de retries ({max_retries}) atingido após erro de requisição.")
                return None # Falha persistente na requisição
    
    print(f"Falha ao obter resposta da API para {base_url} após {max_retries +1} tentativas.")
    return None


def buscar_combinado_doi_fulltext(doi, termo_busca, entity_type="outputs"):
    """
    Busca na API CORE combinando DOI e um termo no fullText para uma entidade específica.
    Retorna (True, entity_id, None) se encontrado, (False, None, error_message) caso contrário.
    """
    if not doi or not termo_busca:
        return False, None, "DOI ou termo de busca está vazio."

    termo_busca_query = termo_busca if (termo_busca.startswith('"') and termo_busca.endswith('"')) else f'"{termo_busca}"'
    doi_query = doi if (doi.startswith('"') and doi.endswith('"')) else f'"{doi}"'
    
    query = f'doi:{doi_query} AND fullText:{termo_busca_query}'
    base_search_url = f"https://api.core.ac.uk/v3/search/{entity_type}"
    api_headers = {"Authorization": f"Bearer {API_KEY}"}
    api_params = {"q": query, "limit": 1}

    print(f"Buscando em '{entity_type}' para DOI: {doi}, Termo FullText: {termo_busca} (Query: {query})")
    
    response = executar_query_api_core(
        base_search_url, 
        api_params, 
        api_headers, 
        MAX_RETRIES_API, 
        INITIAL_BACKOFF_SECONDS_API, 
        MAX_BACKOFF_SECONDS_API
    )

    if response is None: # Falha persistente na requisição
        return False, None, "Falha na comunicação com a API após múltiplas tentativas."
    
    if response.status_code == 200:
        try:
            data = response.json()
            if data.get("total_hits", 0) > 0 and data.get("results") and len(data["results"]) > 0:
                entity_id = data["results"][0].get("id")
                print(f"Termo '{termo_busca}' ENCONTRADO no fullText de '{entity_type}' para DOI {doi}. ID: {entity_id}")
                return True, str(entity_id), None
            else:
                msg = f"Termo '{termo_busca}' NÃO encontrado no fullText de '{entity_type}' para DOI {doi} (hits: {data.get('total_hits', 0)})."
                return False, None, msg
        except ValueError: # Erro ao decodificar JSON
            msg = f"Resposta da API não é JSON válido para {entity_type} DOI {doi}. Conteúdo: {response.text[:200]}"
            print(msg)
            return False, None, msg
    else: # Se a resposta não foi 200 (e não None)
        msg = f"Erro da API {response.status_code} ao buscar {entity_type} para DOI {doi}. Resposta: {response.text[:200]}"
        print(msg)
        return False, None, msg


if __name__ == "__main__":
    try:
        df = None
        codificacoes = ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1']
        for codificacao in codificacoes:
            try:
                df = pd.read_csv(PLANILHA_ENTRADA, encoding=codificacao)
                print(f"Arquivo CSV '{PLANILHA_ENTRADA}' lido com codificação: {codificacao}")
                break
            except UnicodeDecodeError:
                print(f"Falha ao ler CSV com codificação: {codificacao}. Tentando próxima...")
            except FileNotFoundError:
                print(f"ERRO CRÍTICO: Arquivo de entrada não encontrado em: {PLANILHA_ENTRADA}")
                exit()

        if df is None:
            raise Exception(f"Não foi possível decodificar o arquivo CSV: {PLANILHA_ENTRADA} com as codificações testadas.")

        if COLUNA_DOI not in df.columns:
            print(f"ERRO CRÍTICO: Coluna DOI '{COLUNA_DOI}' não encontrada na planilha.")
            exit()
        if COLUNA_TERMO_BUSCA_FULLTEXT not in df.columns:
            print(f"ERRO CRÍTICO: Coluna de termo de busca '{COLUNA_TERMO_BUSCA_FULLTEXT}' não encontrada.")
            exit()

        resultados_finais = []
        total_linhas = len(df)
        print(f"\nIniciando processamento de {total_linhas} linhas da planilha...")

        for index, row in df.iterrows():
            doi_original = row[COLUNA_DOI]
            termo_busca_original = row[COLUNA_TERMO_BUSCA_FULLTEXT]
            
            print(f"\n--- Processando Linha {index + 1}/{total_linhas} ---")
            print(f"DOI: {doi_original}, Termo de Busca: {termo_busca_original}")

            resultado_linha = {
                'doi': doi_original,
                'termo_busca': termo_busca_original,
                'encontrado_em_outputs': False,
                'output_id_encontrado': None,
                'erro_outputs': None,
                'encontrado_em_works': False,
                'work_id_encontrado': None,
                'erro_works': None,
                'status_geral': "Não processado"
            }

            if pd.isna(doi_original) or str(doi_original).strip() == "":
                print("DOI vazio ou NaN. Pulando.")
                resultado_linha['status_geral'] = "Falha: DOI vazio"
                resultado_linha['erro_outputs'] = "DOI vazio"
                # Não precisa definir erro_works se o DOI já é inválido
                resultados_finais.append(resultado_linha)
                continue
            
            if pd.isna(termo_busca_original) or str(termo_busca_original).strip() == "":
                print("Termo de busca vazio ou NaN. Pulando.")
                resultado_linha['status_geral'] = "Falha: Termo de busca vazio"
                resultado_linha['erro_outputs'] = "Termo de busca vazio"
                resultados_finais.append(resultado_linha)
                continue

            doi_str = str(doi_original).strip()
            termo_busca_str = str(termo_busca_original).strip()

            # Busca em Outputs
            encontrado_output, output_id, erro_output = buscar_combinado_doi_fulltext(doi_str, termo_busca_str, "outputs")
            resultado_linha['encontrado_em_outputs'] = encontrado_output
            resultado_linha['output_id_encontrado'] = output_id
            resultado_linha['erro_outputs'] = erro_output if not encontrado_output else None
            
            # Busca em Works
            encontrado_work, work_id, erro_work = buscar_combinado_doi_fulltext(doi_str, termo_busca_str, "works")
            resultado_linha['encontrado_em_works'] = encontrado_work
            resultado_linha['work_id_encontrado'] = work_id
            resultado_linha['erro_works'] = erro_work if not encontrado_work else None

            # Define o status geral com base nos resultados das buscas
            if encontrado_output and encontrado_work:
                resultado_linha['status_geral'] = "Encontrado em Outputs e Works"
            elif encontrado_output:
                resultado_linha['status_geral'] = "Encontrado em Outputs"
            elif encontrado_work:
                resultado_linha['status_geral'] = "Encontrado em Works"
            elif (erro_output and "Falha na comunicação" in erro_output) or \
                 (erro_work and "Falha na comunicação" in erro_work):
                resultado_linha['status_geral'] = "Erro: Falha de comunicação com API"
            elif (erro_output and "API 429" in erro_output) or \
                 (erro_work and "API 429" in erro_work) or \
                 (erro_output and "Too Many Requests" in erro_output) or \
                 (erro_work and "Too Many Requests" in erro_work) : # Verifica se a msg de erro contém indicação de rate limit
                resultado_linha['status_geral'] = "Erro: API Rate Limit Atingido"
            elif erro_output or erro_work: # Outros erros ou não encontrado
                 resultado_linha['status_geral'] = "Não encontrado ou Erro na busca"
            else: # Não encontrado em nenhum, sem erros fatais de API
                resultado_linha['status_geral'] = "Não encontrado"

            resultados_finais.append(resultado_linha)
            
            print(f"Aguardando {DELAY_ENTRE_LINHAS_CSV} segundos antes da próxima linha...")
            time.sleep(DELAY_ENTRE_LINHAS_CSV)

        if resultados_finais:
            resultados_df = pd.DataFrame(resultados_finais)
            colunas_ordenadas = [
                'doi', 'termo_busca', 
                'encontrado_em_outputs', 'output_id_encontrado', 'erro_outputs',
                'encontrado_em_works', 'work_id_encontrado', 'erro_works',
                'status_geral'
            ]
            for col in colunas_ordenadas: # Garante que todas as colunas existam
                if col not in resultados_df.columns:
                    resultados_df[col] = None
            resultados_df = resultados_df[colunas_ordenadas]

            resultados_df.to_csv(ARQUIVO_SAIDA_RESULTADOS, index=False, encoding='utf-8')
            print(f"\nResultados do processamento foram salvos em: {ARQUIVO_SAIDA_RESULTADOS}")
        else:
            print("\nNenhuma linha processada para salvar resultados.")

    except FileNotFoundError:
        pass # Já tratado
    except Exception as e:
        print(f"Ocorreu um erro geral e fatal no script: {e}")
        import traceback
        traceback.print_exc()



Falha ao ler CSV com codificação: utf-8. Tentando próxima...
Arquivo CSV '/work/CenaNoink/planilha30_31_32(Sheet1).csv' lido com codificação: latin-1
ERRO CRÍTICO: Coluna de termo de busca 'TERMO_BUSCA_FULLTEXT' não encontrada.

Iniciando processamento de 503 linhas da planilha...
Ocorreu um erro geral e fatal no script: 'TERMO_BUSCA_FULLTEXT'
Traceback (most recent call last):
  File "/root/venv/lib/python3.11/site-packages/pandas/core/indexes/base.py", line 3791, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "index.pyx", line 152, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 181, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7080, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'TERMO_BUSCA_FULLTEXT'

The above exception was t

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d35fdc8b-8543-45dc-ae25-3ba609dd01b9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>